In [1]:
%pip install --upgrade --quiet  playwright > /dev/null
%pip install --upgrade --quiet  lxml

# If this is your first time using playwright, you'll have to install a browser executable.
# Running `playwright install` by default installs a chromium browser executable.
# playwright install


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 2.8 MB/s eta 0:00:00a 0:00:01
  Using cached soupsieve-2.5-py3-none-any.whl (36 kB)

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [24]:
!pip install -U langchain-google-genai


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [10]:
!playwright install

152.8 MiB [                    ] 0% 0.0s152.8 MiB [                    ] 0% 39.4s152.8 MiB [                    ] 0% 155.1s152.8 MiB [                    ] 0% 132.0s152.8 MiB [                    ] 0% 105.0s152.8 MiB [                    ] 0% 90.1s152.8 MiB [                    ] 0% 128.9s152.8 MiB [                    ] 0% 117.2s152.8 MiB [                    ] 0% 140.9s152.8 MiB [                    ] 0% 134.5s152.8 MiB [                    ] 0% 128.5s152.8 MiB [                    ] 0% 120.7s152.8 MiB [                    ] 0% 114.2s152.8 MiB [                    ] 0% 106.3s152.8 MiB [                    ] 0% 100.0s152.8 MiB [                    ] 0% 95.5s152.8 MiB [                    ] 0% 89.7s152.8 MiB [                    ] 0% 84.8s152.8 MiB [                    ] 0% 81.4s152.8 MiB [                    ] 0% 79.1s152.8 MiB [                    ] 0% 76.4s152.8 MiB [                    ] 0% 73.9s152.8 MiB [                    ] 1% 70.5s152.8 MiB [                    ] 1% 68.8s152.8

In [3]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit

In [5]:
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.\n",      },
)

In [6]:
# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

## Instantiating a Browser Toolkit

In [13]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/none/.cache/ms-playwright/chromium-1097/chrome-linux/chrome> version=121.0.6167.57>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/none/.cache/ms-playwright/chromium-1097/chrome-linux/chrome> version=121.0.6167.57>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/none/.cache/ms-playwright/chromium-1097/chrome-linux/chrome> version=121.0.6167.57>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/none/.cache/ms-playwright/chromium-1097/chrome-linux/chrome> version=121.0.6167.57>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/none/.cache/ms-playwright/chromium-1097/chrome-linux/chrome> version=121.0.6167.57>),
 GetElementsTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/none/.ca

In [14]:
tools_by_name = {tool.name: tool for tool in tools}
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]


In [18]:
await navigate_tool.arun(
    {"url": "https://web.archive.org/web/20230428131116/https://www.cnn.com/world"}
)

TimeoutError: Timeout 30000ms exceeded.

In [19]:
# The browser is shared across tools, so the agent can interact in a stateful manner
await get_elements_tool.arun(
    {"selector": ".container__headline", "attributes": ["innerText"]}
)

'[{"innerText": "These Ukrainian veterinarians are risking their lives to care for dogs and cats in the war zone"}, {"innerText": "Life in the ocean’s ‘twilight zone’ could disappear due to the climate crisis"}, {"innerText": "Clashes renew in West Darfur as food and water shortages worsen in Sudan violence"}, {"innerText": "Thai policeman’s wife investigated over alleged murder and a dozen other poison cases"}, {"innerText": "American teacher escaped Sudan on French evacuation plane, with no help offered back home"}, {"innerText": "Dubai’s emerging hip-hop scene is finding its voice"}, {"innerText": "How an underwater film inspired a marine protected area off Kenya’s coast"}, {"innerText": "The Iranian drones deployed by Russia in Ukraine are powered by stolen Western technology, research reveals"}, {"innerText": "India says border violations erode ‘entire basis’ of ties with China"}, {"innerText": "Australian police sift through 3,000 tons of trash for missing woman’s remains"}, {"in

In [20]:
# If the agent wants to remember the current webpage, it can use the `current_webpage` tool
await tools_by_name["current_webpage"].arun({})

'https://web.archive.org/web/20230428133211/https://cnn.com/world'

## Use within an Agent

In [30]:
from langchain.agents import AgentType, initialize_agent
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key="",temperature=0, convert_system_message_to_human=True)  # or any other LLM, e.g., ChatOpenAI(), OpenAI()

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [32]:
result = await agent_chain.arun("What elements should I scrape for langchain.com?")
print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "get_elements",
  "action_input": {
    "selector": "div.lang-chain-item"
  }
}
```
Observation: []
Thought:I should extract the text from the elements I scraped.

Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```
Observation: LangChain 🦜🔗 LangChain LangSmith LangServe Agents Retrieval Evaluation Blog Docs Partners Catch up on the latest in LangSmith on YouTube. Check it out. Get your LLM application from prototype to production Build context-aware, reasoning applications with LangChain’s flexible abstractions and AI-first toolkit. Go To Docs Contact Sales global corporations, STARTUPS, and TINKERERS build with LangChain A complete set of powerful building blocks Get started fast with our comprehensive library of open-source components and pre-built chains for any use-case. Go To Docs Smart connections to any source of data or knowledge See all integrations Easily-swappable components Set yourself up f